# Agent Demo with Simplified Connection Management

This notebook demonstrates an agent using multiple tool types with simplified connection management:
1. **Local Calculator**: Mathematical expressions via MCP
2. **Brave Search**: Web search via MCP (if API key available)
3. **Think Tool**: Internal reasoning tool

In [1]:
import os
import sys
import nest_asyncio
nest_asyncio.apply()

parent_dir = os.path.dirname(os.getcwd())
sys.path.insert(0, parent_dir)
from agents.agent import Agent, ModelConfig
from agents.tools.think import ThinkTool

# 1. Configure tools and MCP servers

In [2]:
# Standard Python tool
think_tool = ThinkTool()

# Python MCP server
calculator_server_path = os.path.abspath(os.path.join(os.getcwd(), "tools/calculator_mcp.py"))
calculator_server = {
    "type": "stdio",
    "command": "python",
    "args": [calculator_server_path]
}
print(f"Calculator server configured: {'Yes' if calculator_server else 'No'}")


# # Brave MCP server written in TypeScript

# brave_api_key = os.environ.get("BRAVE_API_KEY_BASE_DATA", "")
# print(f"Brave API key available: {'Yes' if brave_api_key else 'No'}")
# brave_search_server = {
#     "type": "stdio",
#     "command": "npx",
#     "args": ["-y", "@modelcontextprotocol/server-brave-search"],
#     "env": {
#         "BRAVE_API_KEY": brave_api_key, 
#         "PATH": f"{os.path.dirname('npx')}:" + os.environ.get("PATH", "")
#     }
# }
# print(f"Brave search server configured: {'Yes' if brave_search_server else 'No'}")



Calculator server configured: Yes


# 2. Initialize agent with combined tools

In [3]:
# Create agent config 
system_prompt = """
You are a helpful assistant with access to:
1. Web search (brave_web_search, brave_local_search)
2. Mathematical calculator (calculate) 
3. A tool to think and reason (think)

Always use the most appropriate tool for each task.
"""


# Initialize agent with standard tools and MCP servers
agent = Agent(
    name="Multi-Tool Agent",
    system=system_prompt,
    tools=[think_tool], 
    mcp_servers=[calculator_server], 
    config=ModelConfig(
        model="qwen-plus", 
        max_tokens=4096,
        temperature=1.0
    ),
    verbose=True
)


[Multi-Tool Agent] Agent initialized


# 3. Run queries

In [4]:
# Example query
agent.run("What's the square root of the OKC population in 2022")

Loaded 1 MCP tools from 1 servers.

[Multi-Tool Agent] Received: What's the square root of the OKC population in 2022
{'content': '', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_d9edeeb32e1a4d189b0c36', 'function': {'arguments': '{"thought":"The approximate population of OKC (Oklahoma City) in 2022 is needed. Without internet access, I\'ll use an estimated figure. Oklahoma City had a population of about 655,000 in 2020. Let\'s assume the population grew by around 1% per year for simplicity. Thus, we can roughly estimate the 2022 population."}', 'name': 'think'}, 'type': 'function', 'index': 0}]}
Error during cleanup: unhandled errors in a TaskGroup (1 sub-exception)


AttributeError: 'dict' object has no attribute 'tool_calls'

In [ ]:
await agent.run_async("How many bananas will fit in an Toyota GR86?")